# 1-5 Transformerの学習・推論、判定根拠の可視化

- TransformerモデルとchABSAのDataLoaderを使用してクラス分類を学習させる。
- テストデータで推論をし、判断根拠となるAttentionを可視化する。


# 事前準備

In [38]:
# パッケージのimport
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext


In [39]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DatasetとDataLoaderを作成

In [46]:
from utils.dataloader import get_chABSA_DataLoaders_and_TEXT
# 読み込み
train_dl, val_dl, TEXT = get_chABSA_DataLoaders_and_TEXT(max_length=256, batch_size=8)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}


In [47]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fcc1a54cac8>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             '0': 4,
             '、': 5,
             'の': 6,
             'は': 7,
             'た': 8,
             'まし': 9,
             '円': 10,
             'に': 11,
             'と': 12,
             '万': 13,
             'が': 14,
             '百': 15,
             'し': 16,
             '％': 17,
             '．': 18,
             '（': 19,
             '）': 20,
             '億': 21,
             'なり': 22,
             'を': 23,
             'で': 24,
             '年度': 25,
             '売上高': 26,
             '連結会計': 27,
             'て': 28,
             '，': 29,
             'により': 30,
             '増': 31,
             '比': 32,
             '減': 33,
             'や': 34,
             '増加': 35,
             '前期比': 36,
             'な': 37,
             '前': 38,
             '前年同期

In [48]:
#データ確認
batch = next(iter(val_dl))
print("="*50)
#単語が数字に変換され、max_lengthに満たない部分はpad=1になっていることが確認できる。
print(batch.Text[0][2])
print("="*50)
print(batch.Label)

tensor([  2,  69,   5, 252,  72,   7,   5, 172,   6,   0,  14, 348,  52, 246,
          5, 112,   7,   0,  30,   0, 186,  23,   0,  11,  16,  28,  78,   5,
        169,  66, 185, 177,   0,  41,   0,  14,   0, 124, 316,  88,  11,  82,
         79,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,  

In [61]:
def create_vocab(pkl_path):
    max_length = 256
    data_path = '/mnt/c/Users/sinfo/Desktop/pytorch/pytorch_advanced-master/django/sample/app1/data'
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False)
    train_ds, val_ds = torchtext.data.TabularDataset.splits(
        path=data_path, train='train.tsv',validation='test.tsv', format='tsv',
        fields=[('Text', TEXT), ('Label', LABEL)])
    japanese_fastText_vectors = Vectors(name='/mnt/c/Users/sinfo/Desktop/pytorch/pytorch_advanced-master/django/sample/app1/data/model.vec')
    # ベクトル化したバージョンのボキャブラリーを作成
    TEXT.build_vocab(train_ds, vectors=japanese_fastText_vectors, min_freq=5)
    pickle_dump(TEXT, pkl_path)
    return TEXT

def pickle_dump(TEXT, path):
    with open(path, 'wb') as f:
        pickle.dump(TEXT, f)

def pickle_load(path):
    with open(path, 'rb') as f:
        TEXT = pickle.load(f)
    return TEXT

In [62]:
TEXT = create_vocab(pkl_path)

In [63]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fcbf3f64fd0>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             '0': 4,
             '、': 5,
             'の': 6,
             'は': 7,
             'た': 8,
             'まし': 9,
             '円': 10,
             'に': 11,
             'と': 12,
             '万': 13,
             'が': 14,
             '百': 15,
             'し': 16,
             '％': 17,
             '．': 18,
             '（': 19,
             '）': 20,
             '億': 21,
             'なり': 22,
             'を': 23,
             'で': 24,
             '年度': 25,
             '売上高': 26,
             '連結会計': 27,
             'て': 28,
             '，': 29,
             'により': 30,
             '増': 31,
             '比': 32,
             '減': 33,
             'や': 34,
             '増加': 35,
             '前期比': 36,
             'な': 37,
             '前': 38,
             '前年同期

In [ ]:
TEXT = pickle_load(pkl_path)

# ネットワークモデルの作成

In [66]:
from utils.transformer import TransformerClassification

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=256, output_dim=2)
print(net)

pe.shape= torch.Size([256, 300])
pe.shape= torch.Size([256, 300])
TransformerClassification(
  (net1): Embedder(
    (embeddings): Embedding(996, 300)
  )
  (net2): PositionalEncoder()
  (net3_1): TransformerBlock(
    (norm_1): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
    (norm_2): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
    (attn): Attention(
      (q_linear): Linear(in_features=300, out_features=300, bias=True)
      (v_linear): Linear(in_features=300, out_features=300, bias=True)
      (k_linear): Linear(in_features=300, out_features=300, bias=True)
      (out): Linear(in_features=300, out_features=300, bias=True)
    )
    (ff): FeedForward(
      (linear_1): Linear(in_features=300, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1)
      (linear_2): Linear(in_features=1024, out_features=300, bias=True)
    )
    (dropout_1): Dropout(p=0.1)
    (dropout_2): Dropout(p=0.1)
  )
  (net3_2): TransformerBlock(
    (norm_1): La

In [7]:
# ネットワークの初期化を定義


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)


# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('ネットワーク設定完了')


ネットワーク設定完了


# 損失関数と最適化手法を定義

In [8]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


# 学習・検証を実施

In [9]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測（dim=1 列方向のＭａｘを取得、predsは最大のindex）

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()   #損失の計算
                        optimizer.step()  # 勾配の更新

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [10]:
# 学習・検証を実行（ＣＰＵで15分程度）
num_epochs = 14
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cpu
-----start-------
Epoch 1/14 | train |  Loss: 0.5539 Acc: 0.7020
Epoch 1/14 |  val  |  Loss: 0.4542 Acc: 0.8078
Epoch 2/14 | train |  Loss: 0.3501 Acc: 0.8492
Epoch 2/14 |  val  |  Loss: 0.4407 Acc: 0.8363
Epoch 3/14 | train |  Loss: 0.2840 Acc: 0.8914
Epoch 3/14 |  val  |  Loss: 0.4546 Acc: 0.8351
Epoch 4/14 | train |  Loss: 0.2457 Acc: 0.8975
Epoch 4/14 |  val  |  Loss: 0.4752 Acc: 0.8280
Epoch 5/14 | train |  Loss: 0.2273 Acc: 0.9132
Epoch 5/14 |  val  |  Loss: 0.4521 Acc: 0.8327
Epoch 6/14 | train |  Loss: 0.2072 Acc: 0.9173
Epoch 6/14 |  val  |  Loss: 0.4715 Acc: 0.8399
Epoch 7/14 | train |  Loss: 0.1995 Acc: 0.9188
Epoch 7/14 |  val  |  Loss: 0.4428 Acc: 0.8458
Epoch 8/14 | train |  Loss: 0.1849 Acc: 0.9228
Epoch 8/14 |  val  |  Loss: 0.4593 Acc: 0.8565
Epoch 9/14 | train |  Loss: 0.1713 Acc: 0.9310
Epoch 9/14 |  val  |  Loss: 0.4266 Acc: 0.8482
Epoch 10/14 | train |  Loss: 0.1515 Acc: 0.9416
Epoch 10/14 |  val  |  Loss: 0.4564 Acc: 0.8517
Epoch 11/14 | train |  Loss:

In [67]:
#torch.save(net_trained.state_dict(), "14_steps_fastText_weight.pth")

param = torch.load('14_steps_fastText_weight.pth')
net_trained.load_state_dict(param)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [68]:
net_trained

TransformerClassification(
  (net1): Embedder(
    (embeddings): Embedding(996, 300)
  )
  (net2): PositionalEncoder()
  (net3_1): TransformerBlock(
    (norm_1): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
    (norm_2): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
    (attn): Attention(
      (q_linear): Linear(in_features=300, out_features=300, bias=True)
      (v_linear): Linear(in_features=300, out_features=300, bias=True)
      (k_linear): Linear(in_features=300, out_features=300, bias=True)
      (out): Linear(in_features=300, out_features=300, bias=True)
    )
    (ff): FeedForward(
      (linear_1): Linear(in_features=300, out_features=1024, bias=True)
      (dropout): Dropout(p=0.1)
      (linear_2): Linear(in_features=1024, out_features=300, bias=True)
    )
    (dropout_1): Dropout(p=0.1)
    (dropout_2): Dropout(p=0.1)
  )
  (net3_2): TransformerBlock(
    (norm_1): LayerNorm(torch.Size([300]), eps=1e-05, elementwise_affine=True)
   

# Attentionの可視化で判定根拠を探る



In [69]:
# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    sentence = batch.Text[0][index]  # 文章
    print("sentenceの形状：", sentence.shape)
    label = batch.Label[index]  # ラベル
    print("labelの形状:", label)
    pred = preds[index]  # 予測
    print("pored:",pred.shape)
    print("sentence:", sentence)
    print(label)

    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    # ラベルと予測結果を文字に置き換え
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '正解ラベル：{}<br>推論ラベル：{}<br><br>'.format(label_str, pred_str)

    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html


In [70]:
from IPython.display import HTML

# Transformerで処理
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

# ミニバッチの用意
batch = next(iter(val_dl))

# GPUが使えるならGPUにデータを送る
inputs = batch.Text[0].to(device)  # 文章
labels = batch.Label.to(device)  # ラベル

print("inputs.shape=",inputs.shape)
# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (inputs != input_pad)
print("input_mask.shape=",input_mask.shape)
#print(inputs)
print(input_mask[0])
# Transformerに入力
outputs, normlized_weights_1, normlized_weights_2 = net_trained(
    inputs, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測


index = 7 # 出力させたいデータ
html_output = mk_html(index, batch, preds, normlized_weights_1,
                      normlized_weights_2, TEXT)  # HTML作成
HTML(html_output)  # HTML形式で出力


inputs.shape= torch.Size([8, 256])
input_mask.shape= torch.Size([8, 256])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.uint8)
sentenceの形状： torch.Size([256])
labelの形状: tensor(0)

# 推論用の1文章をインプットしてラベルとAttentionを可視化する。

In [73]:
def preprocessing_text(text):
    
    # 半角・全角の統一
    text = mojimoji.han_to_zen(text) 
    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)
    text = re.sub('（','', text)
    text = re.sub('）','', text)
    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]+', '0', text)  # 数字

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    return text

# 分かち書き
def tokenizer_mecab(text):
    m_t = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret

# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_mecab(text)  # Janomeの単語分割

    return ret

def create_tensor(text, max_length):
    #入力文章をTorch Teonsor型にのINDEXデータに変換
    token_ids = torch.ones((max_length)).to(torch.int64)
    ids_list = list(map(lambda x: TEXT.vocab.stoi[x] , text))
    print(ids_list)
    for i, index in enumerate(ids_list):
        token_ids[i] = index
    return token_ids


# HTMLを作成する関数を実装


def highlight(word, attn):
    "Attentionの値が大きいと文字の背景が濃い赤になるhtmlを出力させる関数"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(input, preds, normlized_weights_1, normlized_weights_2, TEXT):
    "HTMLデータを作成する"

    # indexの結果を抽出
    index = 0
    sentence = input.squeeze_(0) # 文章  #  torch.Size([1, 256])  > torch.Size([256]) 
    pred = preds[0]  # 予測


    # indexのAttentionを抽出と規格化
    attens1 = normlized_weights_1[index, 0, :]  # 0番目の<cls>のAttention
    attens1 /= attens1.max()

    attens2 = normlized_weights_2[index, 0, :]  # 0番目の<cls>のAttention
    attens2 /= attens2.max()

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 表示用のHTMLを作成する
    html = '推論ラベル：{}<br><br>'.format(pred_str)
  
    # 1段目のAttention
    html += '[TransformerBlockの1段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens1):
        html += highlight(TEXT.vocab.itos[word], attn)
    html += "<br><br>"

    # 2段目のAttention
    html += '[TransformerBlockの2段目のAttentionを可視化]<br>'
    for word, attn in zip(sentence, attens2):
        html += highlight(TEXT.vocab.itos[word], attn)

    html += "<br><br>"

    return html

In [80]:
from IPython.display import HTML, display
from utils.dataloader import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

#インプットデータ
text = "課金売上に関しては、ユーザー数の増加により順調に推移した為、医科セグメントとしては、初の黒字化を達成する事が出来ました"
#textの先頭と末尾に<cls>、<eos>を追加する。
text = tokenizer_with_preprocessing(text)
text.insert(0, '<cls>')
text.append('<eos>')
#   '<cls>': 2, '<eos>': 3,
text = create_tensor(text, 256)
text = text.unsqueeze_(0)   #  torch.Size([256])  > torch.Size([1, 256])

# GPUが使えるならGPUにデータを送る
input = text.to(device)
print("input_shape=",input.shape)
# mask作成
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (input != input_pad)
#print(input)
#print(input_mask)

outputs, normlized_weights_1, normlized_weights_2 = net_trained(input, input_mask)
_, preds = torch.max(outputs, 1)  # ラベルを予測

html_output = mk_html(input, preds, normlized_weights_1, normlized_weights_2, TEXT)  # HTML作成


[2, 0, 59, 0, 7, 5, 686, 326, 6, 35, 30, 189, 11, 43, 16, 8, 0, 5, 0, 68, 116, 7, 5, 0, 6, 0, 23, 0, 52, 0, 14, 0, 9, 8, 3]
input_shape= torch.Size([1, 256])


In [81]:
display(HTML(html_output))

以上